What I cannot create, I do not understand. 내가 만들어낼 수 없다면, 난 그것을 이해하지 못한 것이다

<center>-리처드 파인만-</center>

## Generative Models

- **Generation** : $x_{new} \text{~ } p(x),x_{new}$무언가를 생성하지만 Generative Model은 단순히 무언가를 생성하는 것만이 아니다. sampling


- **Density estimation** : 즉, 해당 이미지가 강아지 같은 것인지 를 판단할수 있는지에 대한 분류의 개념도포함한다. (explicit models라고 한다.) anomaly detection


- **Unsupervised representation learning** : 특성을 파악(꼬리가 있는지, 다리가 있는지) feature learning 이라고 한다


- 생성하는 것만아닌 explicit도 가능하다!

### Basic Discrete Distributions(확률 기본)

Generative Models를 이해하려면 확률에 대한 이론이 필요하다

**Bernoulli distribution** : 0 또는 1, Heads or Tails
- $D$ = {Heads, Tails}


- Specify $P(X = \text{Heads} = p.\text{Then}P(X = \text{Tails}) = 1 - p$ : 1일 확률이 p면 0이 나올 확률은 1-p


- Write : $X$ ~ Ber($p$)

**Categorical distribution** : m개의 주사위를 던짐, m-1개의 parameter필요
- $D = {1,...,m}$
- Specify $P(Y = i) = p_i, \text{such that} \displaystyle\sum^m_{i=1} p_i = 1$
- write : $Y$ ~ Cat($p_1,...,p_m$)


#### 예시

1. RGB는 256 x 256 x 256으로 이루어져 있다.  
    $\rightarrow$이를 표현하려면 256x256x256 - 1개의 parameters필요


2. $X_1 ...X_n$ 개의 binary pixels가 있으면  
    $\rightarrow$$2^n$ 개의 경우의 수가 있고 2n - 1개의 parameters가 필요하다.

### Structure Through Independence
- What if $X_1,...,X_n$ are independent, then
$$p(x,1...,x_n) = P(x_1)p(x_2)\dots p(x_n)$$


- How many possible states?
$$2^n$$


- How many parameters to specify$p(x_1,...,x_n)$?
$$n$$


- $2^n$ entries can be described by just $n$ numbers! But this independence assumption is too strong to model useful distribusions


- 기계학습에서는 parameters가 많을 수록 학습이 어렵다. 


- 그러므로 모든 사용하지 않고 서로 independent 하다고 가정(말이 안되지만)하면 2n 개에서 n 개로 parameters가 줄어든다.

### Conditional Independence
이 적당한 값을 찾기 위해 3가지 rule이 있다

- **Chain rule** 
    - 어떤 가정도 하지 않고 아무것도 바뀐 것이 없다
    - Fully dependent model과 같은 parameter를 가진다.
    - $1+2+2^2+\dots+2^{n-1}=2^n-1$
    - $2^n-1$개의 parameter를 가짐

$$p(x_1,...x_n)=p(x_1)p(x_2|x_1)p(x_3|x_1,x_2)\dots p(x_n|x_1,\dots,x_{n-1})$$



- **Bayes' rule**

$$p(x|y)=\frac{p(x,y)}{p(y)}=\frac{p(y|x)p(x)}{p(y)}$$


- **Conditional Independence**
    - suppose $X_{i+1} \perp X_!,\dots,X_{i-1}|X_i$(    - Markov assumption), then
        - $p(x_1,\dots,x_n) = p(x_1)p(x_2|x_1)\dots p(x_n|x_{n-1})$
    - parameters 는 $2n-1$개이다.
    - Markov assumption으로 parameter를 줄일수 있따
    - 이를 Auto-regressive Model이라 한다.
$$\text{if } x \perp	 y | z, \text{then } p(x|y,z) = p(x|z)$$

## Auto-regressive Model

- 28 x 28 binary pixels가 있다고 하면


- 목표는 $p(x) = p(x_1,...,x_{784}$ over $x\in \{0,1\}^{784}$


- $p(x)$를 어떻게 parametrize(표현?) 할까
    - chain rule 사용        
    - $p(x_{1:784}) = p(x_1)p(x_2|x_1)p(x_3|x_{1:2})\dots$     
    - 이를 autogressive model 이라고 한다
    -  image 는 순서가 지그지그 일수도 행, 열일 수도 있으므로 어떤 식으로 ordering을 할지에 따라 결과가 바뀐다.
    
- 1~n 까지 dependence한 가정도 n-1 ~ n까지의 가정도 모두 Auto-regressive Model이다.


### NADE : Neural Autoregressive Density Estimator

- i번째 pixel을 1~ (i-1) 까지 dependent하게 하는 것
- i번째 pixel에 대해

$$ p(x_i|x_{1:i-1})=\sigma(\alpha_ih_i+b_i)\text{ where }h_i = \sigma(W_{<i}x_{1:i-1}+c)$$

- NADE는 explicit model 이다.
- 즉 generation만 하는게 아닌 Chain-rule을 통해 쪼개면 모든 pixel에 대한 확률을 알 수 있다. 
    - $\{x_1,x_2,\dots,x_{784}\}$에 대해
    - $p(x_{1:784}) = p(x_1)p(x_2|x_1)\dots p(x_784|x_{1:783})\dots$ 이므로
    - $p(x_i|x_{1:i-1})$을 알면 확률을 알 수 있다.


- 예시의 binary가 아닌 continous random variables이면, mixture of Gaussian을 사용할 수 있다.

### Pixel RNN
- RNNs to define auto-regressive model


- $n \times n$ RGM image가 있다고 할시

    $$ p(x)={\prod}^{n^2}_{i=1}p(x_{i,R}|x_{<i})p(x_{i,G}|x_{<i},x_{i,R})p(x_{i,B}|x_{<i},x_{i,R},x_{i,G})$$

- pixel에 대한 RNN을 만듬.


- ordering에 따라 Row LSTM(위쪽 정보 활용), Diagonal BiLSTM(이전 정보들을 다 활용) 으로 나뉜다.

---
## Latent Variable Models

D.Kingma, "Variational Inference and Deep Learning : A New Synthesis, Ph.D.Thesis" 논문을 꼭 읽어보자

#### Question?  
- Is an **autoencoder** a generative model?

### Variational Auto-encoder

- Variational inference(VI)
    - posterior distribution을 찾는 것이 목표
    - posterior distribution(사후확률) 
        - $P_\theta(z|x)$
        
    - Variational distribution : 사후확률은 근사하기 힘들 때 구함
        - $q_\phi(z|x)$
        
    -  posterior distribution과 Variational distribution의 격차를 줄이기 위해선 loss functions인 KL divergence를 줄여야 한다.
    
    
- **How?**
    - posterior error를 모르는 상태에서 Variational distributiont을 잡을순 없다. 
    - 이를 가능하게 해주는 것이 ELBO (trick)이다
    - ELBO를 키우면서 KL divergence를 구할 수 있다
    - $\log p_\theta(D)=\mathbb E_{q_{\phi}(z|x)}[\log \frac{p_{\theta}(x,z)}{q_{\phi}(z|x)}] + D_{KL}(q_{\phi}(z|x)||p_\theta(z|x))$  
    - ELBO : $ \mathbb E_{q_{\phi}(z|x)}[\log \frac{p_{\theta}(x,z)}{q_{\phi}(z|x)}]$를 키운다.
    - 위 식은 Reconstruction Term과 Prior Fitting Term로 나눈다
    
    
- Key limitation 
    - It is an intractable model 
    - The prior fitting term must be differentiable, hence it is hard to use diverse latent prior distributions.
    - In most cases, we use an isotropic Gaussian.

Variational Auto-encoder의 가장 큰 단점은 prior에서 KL divergence를 사용해 Gaussian 사용이 강제됨, 만약 Gaussian을 사용하지 않으려면


### Adversarial Auto-encoder
- It allows us to use any arbitrary latent distributions that we can sample.

## Generative Adversarial Network
`I. Goodfellow et al., "Generative Adversarial Networks", NIPS, 2014`

### GAN
- 위조 지폐범과 $\leftrightarrow$ 위조지폐 검거 경찰의 비유
- 지폐위조를 하고 검증을 하는 과정에서 서로의 실력이 올라감


$$\displaystyle\min_G\displaystyle\max_DV(D,G)=\mathbb E_{x\text~p_{data(x)}}[\log D(x)]+\mathbb E_{z\text~p_z(z)}[\log(1-D(G(z)))]$$

### GAN vs VAE
- GAN은 true/false 학습 후 Real/Fake를 다시 학습

### GAN Objective
- A two player minimax game between generator and discriminator.
- 한명은 높이고 싶어하고, 한명은 낮추고싶어하는 게임에 비유

- For discriminator(판별자):
$$\displaystyle\max_DV(G,D)=\mathbb E_{x\text~p_{data}}[\log D(x)]+\mathbb E_{x\text~p_G}[\log(1-D(x)]$$

- where the optimal discriminator is
$$D^*_G(x) = \frac{p_{data(x)}}{p_{data(x)}+p_{G(x)}}$$

- For generator:
$$\displaystyle\min_GV(G,D)=\mathbb E_{x\text~p_{data}}[\log D(x)]+\mathbb E_{x\text~p_G}[\log(1-D(x)]$$

- Plugging in the optimal discriminator, we get
$$2D_{JSD}[p_{data},p_G]-\log4$$

### GAN의 종류
- DCGAN : image domain을 사용한 gan
- info-GAN : 특정 vector에 집중하게 해줌
- Text2Image : 문장이 주면 image를 반환
- Puzzle-GAN
-**Cycle GAN** : 꼭 알아야한다. Cycle-consistency loss, 말 사진만 주어도 얼룩말 반환
- Star-GAN : 사람 image에 표정, 나이, 성별등을 바꿈
- Progressive-GAN : 고차원 image를 다룬다

In [19]:
import torch
import torch.nn as nn
w = torch.tensor([2.,4.,6.,8.])
nn.init.normal_(w,2,0.1)

tensor([1.9041, 2.1452, 2.0041, 1.9685])